In [50]:
! ls *.py

credentials.py getFEC_v2.py   getPARTY_2.py
getFEC.py      getPARTY.py    run.py


In [51]:
import time
import pandas as pd
import numpy as np


def first_sample(n, sample_size):
    sample_frame = np.random.choice(n, size=n, replace=False, p=None)
    a = np.random.choice(n, size=sample_size, replace=False, p=None)
    return(sample_frame, a)


def resample_one(first_sample):

    sframe = first_sample[0]
    sample = first_sample[1]
    
    sframe = np.setdiff1d(sframe, sample)   
    sample = np.random.choice(sframe, size=1, replace=False, p=None)
    
    return(sframe, sample)


def all_samples(n, first_sample_size=3):        

    s0 = first_sample(n, first_sample_size)
    sr = s0
    count = len(np.atleast_1d(sr[0]))
    samples = [s0[1].tolist()]

    if n == first_sample_size:
        return(samples)
    else:
        pass

    while len(np.atleast_1d(sr[0])) != 1:
        if count == n:
            s1 = resample_one(s0)
            count -=1
            samples.append(s1[1].tolist())
            sr = (s1)
        else:
            sn = resample_one(sr)
            sr = (sn)
            samples.append(sn[1].tolist())     
    else:
        pass

    print(samples)
    return(samples)
    
s = all_samples(10, 1)

rid = None

def get_rid(rid):
    sample_index = 0
    while rid is None:
        sample_index +=1
        rid = s[sample_index][0]
        print(rid)
    else:
        return rid

get_rid(rid)





[[5], [3], [7], [0], [2], [4], [9], [6], [8], [1]]
3


3

In [52]:
s = all_samples(3, 3)
#first_sample(3,3)
s

[[1, 0, 2]]

In [53]:
from credentials import *
import requests
import json, csv
import warnings
import random

from getFEC import *


def get_party_id(committee_id):
    stem = "https://api.open.fec.gov/v1/committee/"
    end = "{}/candidates/?sort=name&page=1&api_key={}&per_page=100".format(committee_id, api_key)
    url = stem+end

    data = get_url(url)['results']
    if len(data) <= 0:
        warnings.warn('WARNING: get_party_id search by committee id returns no results, try another method')
        return ("UNKNOWN PARTY ID")
    else:
        party_id = data[0]['party_full']
        return party_id


def get_committee_details(committee_id):
    stem = "https://api.open.fec.gov/v1/committee/"
    end = "{}/?sort=name&api_key={}&per_page=100&page=1".format(committee_id, api_key)
    url = stem+end

    data = get_url(url)['results']
    party_id = data[0]['party_full']
    candidate_ids = data[0]['candidate_ids']
    #office = data[0]['office_full']
    #name = data[0]['name']
    #return data[0]['party']
    return party_id, candidate_ids

def alt_receipt_id(rid, s, sample_frame):
    if rid == None:
        available = s
        next_sample = random.sample(set(sample_frame))
        r_new = r

def get_schedule_b_receipts(committee_id):
    #TODO way to specify year in URL?
    stem = "https://api.open.fec.gov/v1/committee/{}/schedules/".format(committee_id)
    end = "schedule_b/by_recipient_id/?per_page=20&api_key={}&page=1".format(api_key)
    url = stem+end
    
    #import pdb
    #pdb.set_trace()

    data = get_url(url)['results']
    if len(data) <= 0:
        warnings.warn('WARNING: get_schedule_b_receipts search by committee id returns no results, try another method')
        return None
    elif len(data) > 0 and len(data) < 3:
        return data[0]['recipient_id']
    elif len(data) > 0 and len(data) >= 3:
        
        s = all_samples(len(data), 3)

        def get_rid(rid, sample_index=0):
            while rid is None:
                sample_index +=1
                print(s[sample_index][0])
                rid = data[s[sample_index][0]]['recipient_id']
                print("none", rid)
            else:
                return rid, sample_index
        
        r1 = get_rid(data[s[0][0]]['recipient_id'], 0)
        r2 = get_rid(data[s[0][1]]['recipient_id'], r1[1])
        r3 = get_rid(data[s[0][2]]['recipient_id'], r2[1])
    
        print(r1, r2, r3)
        return [r1[0], r2[0], r3[0]]
    else:
        pass

def find_schedule_b_results(committee_ids):
    if type(committee_ids) is list:
        cids = committee_ids
    else:
        cids = [committee_ids]
    assert len(cids) > 0

    print(cids)
    
    if len(cids) == 1:
        #print(cids)
        party_id = get_committee_details(cids[0])[0]
        return party_id
    else:

        pid_1 = get_committee_details(cids[0])[0]
        pid_2 = get_committee_details(cids[1])[0]
        pid_3 = get_committee_details(cids[2])[0]
        #print(pid_1, pid_2, pid_3)
        if (pid_1 == pid_2 == pid_3) is True:
            party_id = pid_1
            return party_id
        else:
            print("UNCLEAR Schedule B")
            print(pid_1, pid_2, pid_3)
            #return "UNCLEAR Schedule B Party ID"
            return(pid_1, pid_2, pid_3)


def search_party_id(committee_id):

    party_results = get_committee_details(committee_id)

    if party_results[0] is None and len(party_results[1]) == 0:
        print("no party results, further_tests_needed", party_results)
        print("[*] conducting schedule b search...")
        #import pdb
        #pdb.set_trace()
        
        schedule_b_receipts = get_schedule_b_receipts(committee_id)
        party_id = find_schedule_b_results(schedule_b_receipts)
        return party_id

    elif party_results[0] is None and len(party_results[1]) > 0:
        print("candidate id exists but no party results", party_results)
        #try get_party_id search
        party_id = get_party_id(committee_id)
        return party_id

    elif party_results[0] is not None:
        print("party id found in committee details: ", party_results)
        party_id = party_results[0]
        return party_id

    else:
        #print("pass, unknown results", party_results)
        party_id = "UNKNOWN D"
        return party_id

#test_ids = ["C00401224", "C00000935", "C00464297", "C00571380"]
#test_manual_inf = ["dem pac", "dem comm", "rep can", "rep pac"]
#for cid  in test_ids:
#    party_id = search_party_id(cid)
#    print(party_id)


In [55]:
test_ids = ["C00401224", "C00000935", "C00464297", "C00571380", "C00230482", "C00606962"]
#test_ids = ["C00606962"]
#test_ids = ["C00401224"]
test_manual_inf = ["dem pac", "dem comm", "rep can", "rep pac"]
for cid  in test_ids:
    party_id = search_party_id(cid)
    print(party_id)

no party results, further_tests_needed (None, [])
[*] conducting schedule b search...
[[3, 4, 7], [2], [9], [13], [5], [14], [15], [0], [10], [16], [19], [11], [6], [17], [18], [8], [1], [12]]
2
none C00636696
('C00647925', 0) ('C00620021', 0) ('C00636696', 1)
['C00647925', 'C00620021', 'C00636696']
UNCLEAR Schedule B
DEMOCRATIC PARTY None DEMOCRATIC PARTY
('DEMOCRATIC PARTY', None, 'DEMOCRATIC PARTY')
party id found in committee details:  ('DEMOCRATIC PARTY', [])
DEMOCRATIC PARTY
candidate id exists but no party results (None, ['S0NH00235'])
REPUBLICAN PARTY
no party results, further_tests_needed (None, [])
[*] conducting schedule b search...
[[14, 4, 0], [12], [7], [3], [5], [11], [9], [15], [6], [18], [16], [13], [19], [10], [2], [1], [17], [8]]
('C00376087', 0) ('C00067231', 0) ('C00001305', 0)
['C00376087', 'C00067231', 'C00001305']
UNCLEAR Schedule B
REPUBLICAN PARTY None REPUBLICAN PARTY
('REPUBLICAN PARTY', None, 'REPUBLICAN PARTY')
candidate id exists but no party results (Non

In [ ]:
#extract party_id's from getFEC data


In [ ]:
#df = pd.read_csv("2012__Boeing__schedule_a_merged.csv")

In [ ]:
#committee_ids = df.committee_id.drop_duplicates()

In [ ]:
#committee_ids.shape

In [ ]:
#for id in committee_ids:
    

In [ ]:
#for cid  in committee_ids:
#    party_id = search_party_id(cid)
#    time.sleep(1)
#    print(party_id)